In [1]:
import osgeo.ogr as ogr
import osgeo.osr as osr
from osgeo import gdal
import pandas as pd

def csv2shp(csv_path, shp_path, layerName):
    # 解决中文字符问题
    #gdal.SetConfigOption("GDAL_FILENAME_IS_UTF8","NO") 
    #gdal.SetConfigOption("SHAPE_ENCODING","") 
    
    # 设置空间参考,4326代表WGS84
    SpatialReference  = osr.SpatialReference()
    SpatialReference .ImportFromEPSG(4326)
    
    # 新建DataSource,Layer
    driver = ogr.GetDriverByName("ESRI Shapefile")
    data_source = driver.CreateDataSource(shp_path)
    layer = data_source.CreateLayer(layerName, SpatialReference, ogr.wkbPoint)
    
    # 读取csv文件
    csv_df = pd.read_csv(csv_path)
    # csv所有列名,即shp的字段名
    filed_names = list(csv_df)
    
    lat_fld = ogr.FieldDefn('latitude', ogr.OFTReal)
    lat_fld.SetWidth(10)
    lat_fld.SetPrecision(6)
    layer.CreateField(lat_fld)
    
    lon_fld = ogr.FieldDefn('longtitude', ogr.OFTReal)
    lon_fld.SetWidth(10)
    lon_fld.SetPrecision(6)
    layer.CreateField(lon_fld)
    
    sealevel_fld = ogr.FieldDefn('mean_seale', ogr.OFTReal)
    sealevel_fld.SetWidth(10)
    sealevel_fld.SetPrecision(4)
    layer.CreateField(sealevel_fld)
    

    # 从layer中读取相应的feature类型，并创建feature
    featureDefn = layer.GetLayerDefn()
    feature = ogr.Feature(featureDefn)
    
    # 设定几何形状
    point = ogr.Geometry(ogr.wkbPoint)
    
    # 循环输入字段属性值
    for i in range(len(csv_df)):
        
        feature.SetField('latitude', float(csv_df.iloc[i, 0]))
        feature.SetField('longtitude', float(csv_df.iloc[i, 1]))
        feature.SetField('mean_seale', float(csv_df.iloc[i, 2]))
        # 设置几何信息部分
        # 利用经纬度创建点,X为经度,Y为纬度(数据第2列是经度,第1列是纬度)
        point.AddPoint(float(csv_df.iloc[i, 1]), float(csv_df.iloc[i, 0]))
        feature.SetGeometry(point)
        
        # 将feature写入layer
        layer.CreateFeature(feature)
    
    # 从内存中清除 ds，将数据写入磁盘中
    data_source.Destroy()

for i in range(2015, 2017):
    csv_path = r"%s_sealevel.csv"%i
    shp_path = r"%s_sealevel.shp"%i
    csv2shp(csv_path, shp_path, "sealevel")